In [1]:
import pandas as pd
import requests
pd.options.display.max_columns = None

In [2]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
token = os.getenv("MARVEL_KEY")

In [4]:
marvel = requests.get("http://gateway.marvel.com/docs")
marvel.status_code

200

In [5]:
marvel_ = marvel.json()
marvel_

{'swaggerVersion': '1.0',
 'apiVersion': 'Cable',
 'basePath': 'http://gateway.marvel.com',
 'apis': [{'path': '/docs/public', 'description': ''}]}

In [6]:
from marvel import Marvel 

m = Marvel("0b73dfe76602c731dbcadf43f5e9e0fd", token)

En primer lugar, vamos a traer todos los personajes de marvel y almacenarlos en un data frame. Sin e,bargo, nos encontramos múltiples problemas:
1️⃣ El DataFrame inicial contiene mútliples datos que no son de interés, como el code o status.
2️⃣ Los datos interesantes, únicmaente están en la cuarta fila, el resto son datos vacíos.
3️⃣ Se encuentran en diccionario dentro del propio dataframe.
4️⃣ Dentro de ese diccionario hay más diccionarios.

Así, vamos a proceder a extraer la información más importante para cada personaje.

In [7]:
characters = m.characters

characters_all = characters.all()
characters_all

characters_frame = pd.DataFrame(characters_all) 
characters_frame.head(10)

,code,status,copyright,attributionText,attributionHTML,etag,data
count,200,Ok,© 2022 MARVEL,Data provided by Marvel. © 2022 MARVEL,"<a href=""http://marvel.com"">Data provided by M...",8baf3b3ef38ad9175e1d1e0c6ef148e4b632d701,20
limit,200,Ok,© 2022 MARVEL,Data provided by Marvel. © 2022 MARVEL,"<a href=""http://marvel.com"">Data provided by M...",8baf3b3ef38ad9175e1d1e0c6ef148e4b632d701,20
offset,200,Ok,© 2022 MARVEL,Data provided by Marvel. © 2022 MARVEL,"<a href=""http://marvel.com"">Data provided by M...",8baf3b3ef38ad9175e1d1e0c6ef148e4b632d701,0
results,200,Ok,© 2022 MARVEL,Data provided by Marvel. © 2022 MARVEL,"<a href=""http://marvel.com"">Data provided by M...",8baf3b3ef38ad9175e1d1e0c6ef148e4b632d701,"[{'id': 1011334, 'name': '3-D Man', 'descripti..."
total,200,Ok,© 2022 MARVEL,Data provided by Marvel. © 2022 MARVEL,"<a href=""http://marvel.com"">Data provided by M...",8baf3b3ef38ad9175e1d1e0c6ef148e4b632d701,1559


In [8]:
characters_dropped = characters_frame.drop(characters_frame.index[[0,1,2,4]])
characters_dropped.head(2)

,code,status,copyright,attributionText,attributionHTML,etag,data
results,200,Ok,© 2022 MARVEL,Data provided by Marvel. © 2022 MARVEL,"<a href=""http://marvel.com"">Data provided by M...",8baf3b3ef38ad9175e1d1e0c6ef148e4b632d701,"[{'id': 1011334, 'name': '3-D Man', 'descripti..."


In [9]:
def ext_dicc(col):
    '''
    extrae la información necesario de un diccionario
 
    col (str): columna sobre la que aplicamos la función
        
    return: diccionario con toda la información que tenemos
    '''
    try:
        return  col[0]
    except: 
        pass

In [10]:
characters_dropped["Details"] = characters_dropped["data"].apply(ext_dicc)

In [11]:
characters_dropped

,code,status,copyright,attributionText,attributionHTML,etag,data,Details
results,200,Ok,© 2022 MARVEL,Data provided by Marvel. © 2022 MARVEL,"<a href=""http://marvel.com"">Data provided by M...",8baf3b3ef38ad9175e1d1e0c6ef148e4b632d701,"[{'id': 1011334, 'name': '3-D Man', 'descripti...","{'id': 1011334, 'name': '3-D Man', 'descriptio..."


In [12]:
characters_dropped.drop(['etag','code', 'status'], axis = 1)

,copyright,attributionText,attributionHTML,data,Details
results,© 2022 MARVEL,Data provided by Marvel. © 2022 MARVEL,"<a href=""http://marvel.com"">Data provided by M...","[{'id': 1011334, 'name': '3-D Man', 'descripti...","{'id': 1011334, 'name': '3-D Man', 'descriptio..."


In [13]:
characters_completo = pd.DataFrame(characters_dropped.iloc[0,6])
characters_completo.head(2)

,id,name,description,modified,thumbnail,resourceURI,comics,series,stories,events,urls
0,1011334,3-D Man,,2014-04-29T14:18:17-0400,{'path': 'http://i.annihil.us/u/prod/marvel/i/...,http://gateway.marvel.com/v1/public/characters...,"{'available': 12, 'collectionURI': 'http://gat...","{'available': 3, 'collectionURI': 'http://gate...","{'available': 21, 'collectionURI': 'http://gat...","{'available': 1, 'collectionURI': 'http://gate...","[{'type': 'detail', 'url': 'http://marvel.com/..."
1,1017100,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...,2013-09-18T15:54:04-0400,{'path': 'http://i.annihil.us/u/prod/marvel/i/...,http://gateway.marvel.com/v1/public/characters...,"{'available': 4, 'collectionURI': 'http://gate...","{'available': 2, 'collectionURI': 'http://gate...","{'available': 7, 'collectionURI': 'http://gate...","{'available': 0, 'collectionURI': 'http://gate...","[{'type': 'detail', 'url': 'http://marvel.com/..."


In [14]:
comics_character = pd.DataFrame(characters_completo.iloc[0,6])
comics_character.head(2)

,available,collectionURI,items,returned
0,12,http://gateway.marvel.com/v1/public/characters...,{'resourceURI': 'http://gateway.marvel.com/v1/...,12
1,12,http://gateway.marvel.com/v1/public/characters...,{'resourceURI': 'http://gateway.marvel.com/v1/...,12


In [15]:
characters_completo.drop(['thumbnail', 'resourceURI', 'urls'], axis = 1).head(2)

,id,name,description,modified,comics,series,stories,events
0,1011334,3-D Man,,2014-04-29T14:18:17-0400,"{'available': 12, 'collectionURI': 'http://gat...","{'available': 3, 'collectionURI': 'http://gate...","{'available': 21, 'collectionURI': 'http://gat...","{'available': 1, 'collectionURI': 'http://gate..."
1,1017100,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...,2013-09-18T15:54:04-0400,"{'available': 4, 'collectionURI': 'http://gate...","{'available': 2, 'collectionURI': 'http://gate...","{'available': 7, 'collectionURI': 'http://gate...","{'available': 0, 'collectionURI': 'http://gate..."


Es posible buscar los datos concretos de un personaje por su nombre. 
A pesar de ello, la api de marvel debe estar bastante estropeada (o ser una chusta) porque no aparece ni uno de los más conocidos. 🙄

In [16]:
characters_completo[characters_completo["name"] == "Agent Zero"]

,id,name,description,modified,thumbnail,resourceURI,comics,series,stories,events,urls
16,1009150,Agent Zero,,1969-12-31T19:00:00-0500,{'path': 'http://i.annihil.us/u/prod/marvel/i/...,http://gateway.marvel.com/v1/public/characters...,"{'available': 28, 'collectionURI': 'http://gat...","{'available': 9, 'collectionURI': 'http://gate...","{'available': 30, 'collectionURI': 'http://gat...","{'available': 0, 'collectionURI': 'http://gate...","[{'type': 'detail', 'url': 'http://marvel.com/..."


Ahora... realicemos lo mismo con los creadores.

In [18]:
creators = m.creators

creators_all = creators.all()

creators = pd.DataFrame(creators_all)

creators.head(2)

,code,status,copyright,attributionText,attributionHTML,etag,data
count,200,Ok,© 2022 MARVEL,Data provided by Marvel. © 2022 MARVEL,"<a href=""http://marvel.com"">Data provided by M...",c4199c6c0ee8e0b364cf7a5d34d09fdd6956db9e,20
limit,200,Ok,© 2022 MARVEL,Data provided by Marvel. © 2022 MARVEL,"<a href=""http://marvel.com"">Data provided by M...",c4199c6c0ee8e0b364cf7a5d34d09fdd6956db9e,20


In [19]:
creators_dropped = creators.drop(creators.index[[0,1,2,4]])
creators_dropped.head(2)

,code,status,copyright,attributionText,attributionHTML,etag,data
results,200,Ok,© 2022 MARVEL,Data provided by Marvel. © 2022 MARVEL,"<a href=""http://marvel.com"">Data provided by M...",c4199c6c0ee8e0b364cf7a5d34d09fdd6956db9e,"[{'id': 13970, 'firstName': '#O', 'middleName'..."


In [20]:
creators_dropped.drop(['etag','code', 'status'], axis = 1)

,copyright,attributionText,attributionHTML,data
results,© 2022 MARVEL,Data provided by Marvel. © 2022 MARVEL,"<a href=""http://marvel.com"">Data provided by M...","[{'id': 13970, 'firstName': '#O', 'middleName'..."


In [21]:
creators_completo = pd.DataFrame(creators_dropped.iloc[0,6])
creators_completo.head(2)

,id,firstName,middleName,lastName,suffix,fullName,modified,thumbnail,resourceURI,comics,series,stories,events,urls
0,13970,#O,,,,#O,2019-12-11T17:10:07-0500,{'path': 'http://i.annihil.us/u/prod/marvel/i/...,http://gateway.marvel.com/v1/public/creators/1...,"{'available': 4, 'collectionURI': 'http://gate...","{'available': 4, 'collectionURI': 'http://gate...","{'available': 3, 'collectionURI': 'http://gate...","{'available': 0, 'collectionURI': 'http://gate...","[{'type': 'detail', 'url': 'http://marvel.com/..."
1,13971,#X,,,,#X,2019-12-11T17:21:29-0500,{'path': 'http://i.annihil.us/u/prod/marvel/i/...,http://gateway.marvel.com/v1/public/creators/1...,"{'available': 4, 'collectionURI': 'http://gate...","{'available': 4, 'collectionURI': 'http://gate...","{'available': 4, 'collectionURI': 'http://gate...","{'available': 0, 'collectionURI': 'http://gate...","[{'type': 'detail', 'url': 'http://marvel.com/..."


In [22]:
creators_completo.drop(['lastName','middleName', 'firstName', 'suffix'], axis = 1).head(2)

,id,fullName,modified,thumbnail,resourceURI,comics,series,stories,events,urls
0,13970,#O,2019-12-11T17:10:07-0500,{'path': 'http://i.annihil.us/u/prod/marvel/i/...,http://gateway.marvel.com/v1/public/creators/1...,"{'available': 4, 'collectionURI': 'http://gate...","{'available': 4, 'collectionURI': 'http://gate...","{'available': 3, 'collectionURI': 'http://gate...","{'available': 0, 'collectionURI': 'http://gate...","[{'type': 'detail', 'url': 'http://marvel.com/..."
1,13971,#X,2019-12-11T17:21:29-0500,{'path': 'http://i.annihil.us/u/prod/marvel/i/...,http://gateway.marvel.com/v1/public/creators/1...,"{'available': 4, 'collectionURI': 'http://gate...","{'available': 4, 'collectionURI': 'http://gate...","{'available': 4, 'collectionURI': 'http://gate...","{'available': 0, 'collectionURI': 'http://gate...","[{'type': 'detail', 'url': 'http://marvel.com/..."


In [23]:
comics_creator = pd.DataFrame(creators_completo.iloc[0,9])
comics_creator.head(2)

,available,collectionURI,items,returned
0,4,http://gateway.marvel.com/v1/public/creators/1...,{'resourceURI': 'http://gateway.marvel.com/v1/...,4
1,4,http://gateway.marvel.com/v1/public/creators/1...,{'resourceURI': 'http://gateway.marvel.com/v1/...,4


🙃 Por último, intento hacer un join para ver si existe alguna relación entre los autores de los comics y los personajes... 
Una vez más, el api de 💩 me manda a peinarme porque no existen coincidencias

In [24]:
marvel_completo = comics_character.merge(comics_creator, on='collectionURI')
marvel_completo

,available_x,collectionURI,items_x,returned_x,available_y,items_y,returned_y
